<div class="row">
  <div class="column">
    <img src="./img/logo-onera.png" width="200">
  </div>
  <div class="column">
    <img src="./img/logo-ISAE_SUPAERO.png" width="200">
  </div>
</div>

# FAST-OAD Postprocessing Tutorial
FAST-OAD is a framework for performing rapid Overall Aircraft Design. The computational core of FAST-OAD is based on the  [OpenMDAO framework](https://openmdao.org/).



This notebook will show you the basic postprocessing features that FAST-OAD offers.

**Note: The 01_tutorial notebook has to be ran to generate the results files used in this postprocessing notebook**

*Tip: hover your mouse pointer over any plot to inspect it or get menu to manipulate it*

In [1]:
import os.path as pth
from fastoad.io import VariableIO
from fastoad.gui.analysis_and_plots import wing_geometry_plot, aircraft_geometry_plot
from fastga.utils.postprocessing.analysis_and_plots import mass_breakdown_sun_plot, mass_breakdown_bar_plot

# For using all screen width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

Failed to import module fastga.models.propulsion.fuel_propulsion.basicIC_engine.unitary_tests.test_openmdao_engine.py
Failed to import module fastga.models.performances.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.load_analysis.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.geometry.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.weight.cg.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.weight.cg.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.geometry.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.weight.mass_breakdown.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.loops.unitary_tests.test_wing_area.py
Failed to import module fastga.models.handling_qualities.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.performances.unitary_tests.test_cirrus_sr22.py
Failed to import module fastg

To illustrate the set of postprocessing plots we will use the three designs obtained in the `01_tutorial` notebook:
- CeRAS 2750 nm Multidisciplinary Design Analysis (MDA)
- CeRAS 800 nm Multidisciplinary Design Analysis (MDA)
- CeRAS 2750 nm Multidisciplinary Design Optimization (MDO)

In [2]:
DATA_FOLDER_PATH = 'data'
WORK_FOLDER_PATH = 'workdir'

Beechcraft_800nm_MDA_OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs_Beechcraft_800nm_mda.xml')
Beechcraft_1000nm_MDA_OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs_Beechcraft_1000nm_mda.xml')
Beechcraft_800nm_MDO_OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs_Beechcraft_800nm_mdo.xml')

## 1. Introduction

The general philosophy of the postprocessing plots is to use the result data file as the source information. Hence, offline of the computational process. Furthermore, the idea is to compare design results by enabling the superposition of different plots for the postprocessing plots where comparison is relevant.

## 2. Geometry plots
For instance here, we use the `fig` variable generated by the code line 1 and plot the result of line 2 on the existing figure. There is no number of maximum superpositions.

In [ ]:
fig = wing_geometry_plot(Beechcraft_800nm_MDA_OUTPUT_FILE, name='Beechcraft 800 nm MDA')
fig = wing_geometry_plot(Beechcraft_1000nm_MDA_OUTPUT_FILE, name='Beechcraft 1000 nm MDA', fig=fig)
fig = wing_geometry_plot(Beechcraft_800nm_MDO_OUTPUT_FILE, name='Beechcraft 800 nm MDO', fig=fig)
fig.show()

In [ ]:
fig = aircraft_geometry_plot(Beechcraft_800nm_MDA_OUTPUT_FILE, name='Beechcraft 800 nm MDA')
fig = aircraft_geometry_plot(Beechcraft_1000nm_MDA_OUTPUT_FILE, name='Beechcraft 1000 nm MDA', fig=fig)
fig = aircraft_geometry_plot(Beechcraft_800nm_MDO_OUTPUT_FILE, name='Beechcraft 800 nm MDO', fig=fig)
fig.show()

## 3. Mass breakdown plots

This first mass breakdown plot enables to visualize the Maximum TakeOff Weight (MTOW) and Overall Weight Empty (OWE) for a single design.

In [ ]:
fig = mass_breakdown_sun_plot(Beechcraft_800nm_MDA_OUTPUT_FILE)
fig.show()

This second mass breakdown plot provides less detail but enables to compare designs.

## 4. Mission plots

In [ ]:
fig = mass_breakdown_bar_plot(Beechcraft_800nm_MDA_OUTPUT_FILE, name='Beechcraft 800 nm MDA')
fig = mass_breakdown_bar_plot(Beechcraft_1000nm_MDA_OUTPUT_FILE, name='Beechcraft 1000 nm MDA', fig=fig)
fig = mass_breakdown_bar_plot(Beechcraft_800nm_MDO_OUTPUT_FILE, name='Beechcraft 800 nm MDO', fig=fig)
fig.show()

An other post-processing tool is available: the mission viewer. You simply add the mission files that interest you:

In [6]:
from fastoad.gui.mission_viewer import MissionViewer 

Beechcraft_800nm_MDA_MISSION_FILE = pth.join(WORK_FOLDER_PATH, 'workdir/mda_mission_results.csv')

mission = MissionViewer()
mission.add_mission(Beechcraft_800nm_MDA_MISSION_FILE, name = "Beechracft 800 nm MDA")

mission.missions["Beechracft 800 nm MDA"]

,time,altitude,ground_distance,mass,true_airspeed,equivalent_airspeed,mach,density,gamma,alpha,cl_wing,cl_htp,thrust (N),thrust_rate,tsfc (kg/s/N),name
0,0.000000,15.240000,0.000000e+00,1707.057964,44.832452,44.799836,0.131768,1.223200,7.794406,8.980045,0.845089,-0.012050,3539.859018,0.845412,0.000005,sizing:main_route:climb
1,7.950000,63.576994,3.531251e+02,1706.913332,44.936130,44.799836,0.132145,1.217531,7.711520,8.984674,0.845478,-0.012143,3516.085490,0.845389,0.000005,sizing:main_route:climb
2,15.900000,111.513697,7.071365e+02,1706.769563,45.039294,44.799836,0.132520,1.211930,7.629655,8.989226,0.845861,-0.012236,3492.597986,0.845352,0.000005,sizing:main_route:climb
3,23.850000,159.053424,1.062029e+03,1706.626643,45.141947,44.799836,0.132894,1.206395,7.548794,8.993702,0.846238,-0.012328,3469.392189,0.845302,0.000005,sizing:main_route:climb
4,31.800000,206.199463,1.417797e+03,1706.484560,45.244086,44.799836,0.133266,1.200924,7.468925,8.998104,0.846608,-0.012418,3446.464171,0.845238,0.000005,sizing:main_route:climb
5,39.750000,252.955074,1.774436e+03,1706.343300,45.345713,44.799836,0.133636,1.195518,7.390032,9.002434,0.846973,-0.012508,3423.810068,0.845162,0.000005,sizing:main_route:climb
6,47.700000,299.323489,2.131940e+03,1706.202851,45.446828,44.799836,0.134004,1.190175,7.312102,9.006692,0.847331,-0.012596,3401.426077,0.845073,0.000005,sizing:main_route:climb
7,55.650000,345.307917,2.490304e+03,1706.063199,45.547431,44.799836,0.134371,1.184894,7.235120,9.010880,0.847683,-0.012683,3379.308457,0.844972,0.000005,sizing:main_route:climb
8,63.600000,390.911535,2.849523e+03,1705.924332,45.647522,44.799836,0.134736,1.179674,7.159073,9.014999,0.848030,-0.012769,3357.453524,0.844859,0.000005,sizing:main_route:climb
9,71.550000,436.137499,3.209591e+03,1705.786239,45.747101,44.799836,0.135100,1.174515,7.083947,9.019051,0.848371,-0.012855,3335.857654,0.844735,0.000005,sizing:main_route:climb


Or you can plot the missions:

In [7]:
mission.display()